# Ray.io optimization framework

## Search Algorithm

The simplest form of search algorithms are the **GridSearch** and **RandomSearch**. More recent research in this direction have lead to the discovery of more sophisticated algos, including **BayesOptSearch**, **OptunaSearch**, and **HEBOSearch**.

They come as external packages of Ray\[Tune\], directly integrated into

## On a real-world use-case

In addition to the previous components:
* `tune.suggest` The Search Algorithm will direct the search on facts
* `tune.reporter` The Reporter
* `tune.logger`
* `tune.stopper`

We will apply all of the components on an example of GNN for the considered use-case.

Open the file `graphs.py` and debug it.

In [1]:
!cat graphs.py

import numpy as np
import os.path as osp
from pytorch_lightning import LightningDataModule
import torch
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from typing import List, Tuple

from kosmoss import CONFIG, DATA_PATH, METADATA
from kosmoss.dataproc.flows import BuildGraphsFlow


class GNNDataset(Dataset):
    
    def __init__(self) -> None:
        
        self.timestep = str(CONFIG['timestep'])
        self.params = METADATA[str(self.timestep)]['features']
        self.num_shards = self.params['num_shards']
        
        super().__init__(DATA_PATH)

    @property
    def raw_file_names(self) -> list:
        return [""]

    @property
    def processed_file_names(self) -> List[str]:
        return [osp.join(f"graphs-{self.timestep}", f"data-{shard}.pt") 
                for shard in np.arange(self.num_shards)]
    
    
    def download(self) -> None:
        raise Exception("Execute the Notebooks in this Bootcamp following the order de

In [2]:
!cat graphnet.py

from pytorch_lightning import LightningModule
import torch
from torch_geometric.data import Batch
from torch_geometric.nn.models import GAT
import torch_optimizer as optim
import torchmetrics.functional as F
from typing import Dict, List, Union


class LitGAT(LightningModule):
    
    def __init__(self, **kwargs) -> None:
        super().__init__()
        self.save_hyperparameters()
        self.lr = kwargs.pop('lr')
        self.net = GAT(**kwargs)

    def forward(self, 
                x: torch.Tensor, 
                edge_index: torch.Tensor) -> torch.Tensor:
        
        return self.net(x, edge_index)
    
    def _common_step(self, 
                     batch: Batch, 
                     batch_idx: int, 
                     stage: Union['train', 'val', 'test'] = "train") -> List[torch.Tensor]:
        
        y_hat = self(batch.x, batch.edge_index)
        loss = F.mean_squared_error(y_hat, batch.y)
        self.log(f"{stage}_loss", loss, prog_bar=True, on_step=True)
  

In [3]:
!cat runtune.py

import json
import numpy as np
import os
import os.path as osp
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.accelerators import Accelerator
from pytorch_lightning.callbacks.base import Callback
from pytorch_lightning.loggers.wandb import WandbLogger
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.integration.wandb import WandbLoggerCallback
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.suggest.hebo import HEBOSearch
import torch
from typing import List, Union

from kosmoss import ARTIFACTS_PATH, LOGS_PATH
from kosmoss.hyperopt.data import LitGNNDataModule
from kosmoss.hyperopt.models import LitGAT


def main() -> None:
    
    project = os.environ['wandb_project']
    
    def train_gnns(config: dict,
                   num_epochs: int = 10,
                   num_gpus: int = 0) -> None:
        
        